In [103]:
# %pip install git+https://github.com/tensorflow/docs

import os.path
from typing import List

import numpy as np
import pandas as pd

from datetime import date, datetime, timedelta
from thefuzz import fuzz

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_colwidth = 160

from utils.configuration import settings
from utils.utils import *
from tft.api import *


# Config

In [104]:
API_KEY: str = settings.api_key
ASSETS_DIR: str = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT: int = settings.max_count
LATEST_RELEASE = '12.13.453.3037' # '12.12.450.4196' '12.13.453.3037' Version 12.12.448.6653 12.11.446.9344 Version 12.13.453.3037 (Jul 11 2022/18:39:20) [PUBLIC] <Releases/12.13>
RANKED_ID = 1100    # 1090 normal game 1100 ranked game
PATCH: date = date(2022, 7, 16)  # date(2022, 7, 1) date(2022, 7, 16)

TARGETNAME = 'placement'

In [105]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [106]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,htDBsWr4bg_iWFzpFXgtwqtL7DWRHQLfaHeQaVcGN9zJzVA,yCRdMPaSlHwaK_yfsckkubeqkN6xkyKKdm_eIwo51s8vRg,CtkdblTTrHBitusl8hmjrpLeF8CFekHiPDoN7mPWxurVhZKdzemOVYFJw7lFZ5pUyda5JHkoQPo3nQ,novokane,3009,1658044332000,120,htDBsWr4bg_iWFzpFXgtwqtL7DWRHQLfaHeQaVcGN9zJzVA,novokane,345,I,100,77,False,False,True,False
1,ydNv8inPkFpOn7x1i-_9YPCDqjwtTKahLwnAi94XerVFQzA,bTSyw1cDaC-X1cfqnLgfWDU5nYdcVRau4XIFB128q2WgXw,E2oMSgyB3q_7GUukxgMcTUfNw4ZKgwoK09eNmKGc7A2iKpOt0a3PTvS3CU-oj42YcxekDwQVnk1UGQ,Bulla Domini,4904,1658029983000,145,ydNv8inPkFpOn7x1i-_9YPCDqjwtTKahLwnAi94XerVFQzA,Bulla Domini,268,I,144,119,False,False,True,True
2,80GcVsTxXyfZR0m8vwa5lQ0oy7Kmb8ozBRLNC0Df1HkTS9o,j7oX-kfOxU4ERhhrxISwBCKO06WxRsVDOF5JgmnS5l1BTg,xJnpJMivaVcMBZyDh5fvBP1hYr9kf-BZU2uYB-8rAyuFQZUpzbIHcOvaecQHpq5Uj-pBkYG6phIqjg,lizste,4414,1657912812000,115,80GcVsTxXyfZR0m8vwa5lQ0oy7Kmb8ozBRLNC0Df1HkTS9o,lizste,440,I,83,39,False,False,True,True
3,c0AgyF6EUF3KaXeHxzQm6eL1_qIdbwz9B6kR41l85Zvmkd4,5FYjwCpzKu2qFdhNAZjGWrHOuqEiAm-t5u5awkmcw2rH1DA,repXa_Fy2KcjBQP68miyr-cSQ_MDDnrtzKrrFYZxZt-SGikiKJGan8VkEO38qKVh4x5KX3g2EsJDnQ,botlane kazekage,4404,1658104482000,259,c0AgyF6EUF3KaXeHxzQm6eL1_qIdbwz9B6kR41l85Zvmkd4,botlane kazekage,233,I,94,55,False,False,True,False
4,1g61Q4eh4pwpMms3FKtKn1EV23CH5y42KsY1xFRjaxyKmpE,SubHYHii-tUuPcO8KGdsm62zjtAIywSjOvOkbKxQ2Sfb4iw,b5CTtxvLGf_Zsyt7tGMV4yizLxZRbTTZVT-nweVmJiMghoZFyL81fwn-vrTQH88WEFN6581oOwmo8Q,Strategies,4794,1657973074171,391,1g61Q4eh4pwpMms3FKtKn1EV23CH5y42KsY1xFRjaxyKmpE,Strategies,551,I,240,208,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,GVyUTZgWDRO5ULApoOS4dDzdAE4yt8wyH6zYmNy8e2gpDXI,Y9nWhlRPphz4iTq8xKCOuhAp0kPlOb9ogMQfCNqfXW9tvYo,I8Uh0wZcFWqzp4jN50UpSjnzQovBHbl7Apci-NiGJTAfFDBtuhVsDv_q8ZCEmcVWxy9_EpkqzksuKw,moerus,4804,1658101191000,129,GVyUTZgWDRO5ULApoOS4dDzdAE4yt8wyH6zYmNy8e2gpDXI,moerus,464,I,147,102,False,False,True,False
496,Je1-OSJir7hbZG_ll4cEfBVsY3yik57RyqeRjI_u_q4FmtQ,EqaYqlbBjSudwsK4PPVYJLmmsPBtnPxOHHEBAW4-EsHr9AE,8Rv6zj9cB_Fsy3ypWndSvyzwWNAYXl9X__peDzYv7duhpmU-jORNSQ6LzlW9GMszF6cHej2evLh9Ug,Xiao Mei Bi,5415,1657954923000,303,Je1-OSJir7hbZG_ll4cEfBVsY3yik57RyqeRjI_u_q4FmtQ,Xiao Mei Bi,407,I,182,160,False,False,False,False
497,nmojHZ2Uexb0IA-TVcQqoHDxD-2yAjJqKDprxAE2YOm3IsQ,wbMYPQDM0XrCF7_l8BfGSQhLS5kFv2S_5ahiOyGCc88VRW0,z7d7OUj7Eb33vccmOvpoGhgwzLL6XHn9ZVOT1fgd70PTstHwrOB9_k0oFACeUcFGG0eVaHKJsWj9pQ,Bians Big Beefy,685,1658023849000,142,nmojHZ2Uexb0IA-TVcQqoHDxD-2yAjJqKDprxAE2YOm3IsQ,Bians Big Beefy,336,I,144,99,False,False,False,False
498,6-oKkjllJT-X2Aw0qdqqd4M067ZBsY8YavdZGAVjPc04eSk,9C8L3jMgKSKE-cJz8SSloWZBI6SIsYsl1ulNOjoku16z4jI,gVSduaEwvxLN-v2svyuVLpu0LGcQiMcKKcKS1EHM99hsoyfvtrTb1fcg4MHoLrTtnUEokrGw0GvqdQ,Cody Done,4548,1658108020000,337,6-oKkjllJT-X2Aw0qdqqd4M067ZBsY8YavdZGAVjPc04eSk,Cody Done,234,I,136,124,False,False,True,False


# Load unique matches id

In [107]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [108]:
len(uniq_matches_id)

19296

In [109]:
len(matches_asset)

22315

In [110]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [111]:
len(uniq_matches)

19296

## Filter by LATEST_RELEASE version and RANKED_ID game.

In [112]:
matches_asset[-1]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [113]:
matches_asset[-1]['info']['queue_id']

1100

In [114]:
latest_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version']) and (
    RANKED_ID == match['info']['queue_id'])]


In [115]:
latest_patch_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                        and (PATCH <= date.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [116]:
timedelta(days=3)

datetime.timedelta(days=3)

In [117]:
latest_3d_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                     and ((datetime.now() - timedelta(days=3)) <= datetime.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [118]:
len(latest_matches)

4837

In [119]:
len(latest_patch_matches)

3159

In [120]:
len(latest_3d_matches)

3177

# Process api details to datasets rows

In [121]:
def process_matches(df) -> List:
    matches_array = []

    for match_row in df:
        match_id = match_row['metadata']['match_id']

        for participant in match_row['info']['participants']:
            match = {}
            match['match_id'] = match_id
            # match['level'] = participant['level']
            match['placement'] = participant['placement']
            # match['players_eliminated'] = participant['players_eliminated']
            # match['total_damage_to_players'] = participant['total_damage_to_players']

            for augment_index, augment in enumerate(participant['augments']):
                # if augment == 'TFT7_Augment_GuildLootHR':
                #     augment = 'TFT7_Augment_BandOfThieves1'
                match[f'augment{augment_index}'] = augment

            for trait_index, trait in enumerate(participant['traits']):
                match[f'{trait["name"]}'] = trait["tier_current"]

            for unit_index, unit in enumerate(participant['units']):
                match[f'{unit["character_id"]}'] = unit["tier"]
                match['TFT7_TrainerDragon_item1'] = 'None'
                match['TFT7_TrainerDragon_item2'] = 'None'
                for item_index, item in enumerate(unit['itemNames']):
                    match[f'{unit["character_id"]}_item{item_index}'] = item.split('_')[-1]

            matches_array.append(match)

    return matches_array


In [122]:
matches_array = process_matches(latest_matches)
matches_patch_array = process_matches(latest_patch_matches)
matches_3d_array = process_matches(latest_3d_matches)

In [123]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4377502704',
 'placement': 5,
 'augment0': 'TFT7_Augment_AssassinEmblem',
 'augment1': 'TFT6_Augment_RadiantRelics',
 'augment2': 'TFT6_Augment_Diversify2',
 'Set7_Assassin': 3,
 'Set7_Bruiser': 1,
 'Set7_Dragon': 1,
 'Set7_Guild': 1,
 'Set7_Legend': 0,
 'Set7_Ragewing': 0,
 'Set7_Scalescorn': 0,
 'Set7_Shimmerscale': 0,
 'Set7_Tempest': 1,
 'Set7_Whispers': 2,
 'TFT7_Kayn': 2,
 'TFT7_TrainerDragon_item1': 'None',
 'TFT7_TrainerDragon_item2': 'None',
 'TFT7_Kayn_item0': 'LocketOfTheIronSolari',
 'TFT7_Qiyana': 1,
 'TFT7_Diana': 2,
 'TFT7_Diana_item0': 'FrozenHeart',
 'TFT7_Talon': 2,
 'TFT7_Talon_item0': 'Bloodthirster',
 'TFT7_Talon_item1': 'MadredsBloodrazor',
 'TFT7_Talon_item2': 'RunaansHurricane',
 'TFT7_DragonPurple': 2,
 'TFT7_DragonPurple_item0': 'RapidFirecannonRadiant',
 'TFT7_DragonPurple_item1': 'AssassinEmblemItem',
 'TFT7_DragonPurple_item2': 'UnstableConcoction',
 'TFT7_Ornn': 2,
 'TFT7_Pyke': 1,
 'TFT7_Pyke_item0': 'Zephyr',
 'TFT7_Pyke_item1': 'SpearO

In [124]:
matches_league_df = pd.json_normalize(matches_array)
matches_league_patch_df = pd.json_normalize(matches_patch_array)
matches_league_3d_df = pd.json_normalize(matches_3d_array)

In [125]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bruiser,Set7_Dragon,Set7_Guild,Set7_Legend,...,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_LeeSin_item2,TFT7_TahmKench_item2,TFT7_Aatrox_item0,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Taric_item2,TFT7_Twitch_item2,augment3
0,NA1_4377502704,5,TFT7_Augment_AssassinEmblem,TFT6_Augment_RadiantRelics,TFT6_Augment_Diversify2,3.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4377502704,3,TFT6_Augment_CyberneticUplink2,TFT6_Augment_BandOfThieves2,TFT6_Augment_CelestialBlessing2,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4377502704,1,TFT6_Augment_CyberneticShell2,TFT6_Augment_GrandGambler,TFT6_Augment_CelestialBlessing2,NaN,0.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4377502704,6,TFT6_Augment_CyberneticUplink2,TFT7_Augment_BestFriends3,TFT7_Augment_SwiftshotPressTheAttack,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4377502704,4,TFT7_Augment_PandorasBench,TFT7_Augment_CannoneerEmblem2,TFT6_Augment_ThrillOfTheHunt2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38691,NA1_4374334390,4,TFT6_Augment_TargetDummies,TFT7_Augment_LategameSpecialist,TFT6_Augment_ThrillOfTheHunt1,NaN,0.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38692,NA1_4374334390,6,TFT6_Augment_PortableForge,TFT6_Augment_Distancing,TFT7_Augment_BandOfThieves1,1.0,1.0,NaN,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38693,NA1_4374334390,5,TFT7_Augment_UrfsGrabBag1,TFT6_Augment_TinyTitans,TFT6_Augment_CyberneticImplants1,1.0,1.0,NaN,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38694,NA1_4374334390,3,TFT7_Augment_GadgetExpert,TFT6_Augment_Recombobulator,TFT6_Augment_SecondWind1,NaN,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bruiser,Set7_Dragon,Set7_Guild,Set7_Legend,...,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_LeeSin_item2,TFT7_TahmKench_item2,TFT7_Aatrox_item0,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Taric_item2,TFT7_Twitch_item2,augment3
0,NA1_4377502704,5,TFT7_Augment_AssassinEmblem,TFT6_Augment_RadiantRelics,TFT6_Augment_Diversify2,3.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4377502704,3,TFT6_Augment_CyberneticUplink2,TFT6_Augment_BandOfThieves2,TFT6_Augment_CelestialBlessing2,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4377502704,1,TFT6_Augment_CyberneticShell2,TFT6_Augment_GrandGambler,TFT6_Augment_CelestialBlessing2,NaN,0.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4377502704,6,TFT6_Augment_CyberneticUplink2,TFT7_Augment_BestFriends3,TFT7_Augment_SwiftshotPressTheAttack,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4377502704,4,TFT7_Augment_PandorasBench,TFT7_Augment_CannoneerEmblem2,TFT6_Augment_ThrillOfTheHunt2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38691,NA1_4374334390,4,TFT6_Augment_TargetDummies,TFT7_Augment_LategameSpecialist,TFT6_Augment_ThrillOfTheHunt1,NaN,0.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38692,NA1_4374334390,6,TFT6_Augment_PortableForge,TFT6_Augment_Distancing,TFT7_Augment_BandOfThieves1,1.0,1.0,NaN,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38693,NA1_4374334390,5,TFT7_Augment_UrfsGrabBag1,TFT6_Augment_TinyTitans,TFT6_Augment_CyberneticImplants1,1.0,1.0,NaN,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38694,NA1_4374334390,3,TFT7_Augment_GadgetExpert,TFT6_Augment_Recombobulator,TFT6_Augment_SecondWind1,NaN,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sort and reorder columns

In [127]:
def reorder_df_col(df):
    fixed_cols = ['placement', 'match_id', 'augment0', 'augment1', 'augment2']
    all_cols = df.columns
    to_sort_cols = list(set(all_cols) - set(fixed_cols))

    return df.reindex(columns=fixed_cols + sorted(to_sort_cols))


In [128]:
matches_league_df = reorder_df_col(matches_league_df)
matches_league_patch_df = reorder_df_col(matches_league_patch_df)
matches_league_3d_df = reorder_df_col(matches_league_3d_df)

# Output dataframes

In [129]:
# matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)
# matches_league_patch_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))
matches_league_patch_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.csv'), index=False)
matches_league_3d_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{(datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d")}_matches.pickle'))
matches_league_3d_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{(datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d")}_matches.csv'), index=False)

# End